# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## To do
- Loop over wind direction
    - Parallelise using PQDM
- Compare optimals to lossless and phrase wake steering as recovering % of wake losses
- Assess difference in speed of calculation and optimal values when using different fidelity for final optimisation
- Assess results
    - Sensitivity to wind direction/speed
    - Variation in sensitivity between turbines
    - Increase number of turbines

## Setup

In [ ]:
# import libraries
import logging

import numpy as np
import xarray as xr

import utils

logging.basicConfig(
    format="%(asctime)s:%(levelname)s:%(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

## Baseline values

In [ ]:
# extract probabilities for full wind speed/direction range
_, Sector_frequency, P = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=utils.WS_DEFAULT,
    wd=utils.WD_DEFAULT,
)

In [ ]:
# drop wind speeds below cut in speed
ind_cut_in = np.argmax(utils.wfm_low.windTurbines.power(utils.WS_DEFAULT) > 0)
ws = utils.WS_DEFAULT[ind_cut_in:]

In [ ]:
# run baseline simulations
print("--- Lossless ---")
sim_res_ref_lossless, _, _ = utils.run_sim(
    wfm=utils.wfm_lossless,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=ws,
    wd=utils.WD_DEFAULT,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)

print("\n--- Low Fidelity ---")
sim_res_ref_low, _, _ = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=ws,
    wd=utils.WD_DEFAULT,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)

print("\n--- High Fidelity ---")
sim_res_ref_high, _, _ = utils.run_sim(
    wfm=utils.wfm_high,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=ws,
    wd=utils.WD_DEFAULT,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)

## Optimise across for a single wind direction

In [ ]:
# define constants
wd = 270
yaw_shape = (len(sim_res_ref_low.wt), 1, len(sim_res_ref_low.ws))

In [ ]:
# initialise optimal yaw dataset
yaw_opt = xr.Dataset(
    coords={
        "wt": list(sim_res_ref_low.wt.values),
        "wd": [wd],
        "ws": list(sim_res_ref_low.ws.values),
    },
)

In [ ]:
# run optimisation and save output
yaw_opt_power, yaw_opt_lcoe = utils.optimise_direction(
    wd=wd,
    sim_res_ref_low=sim_res_ref_low,
    sim_res_ref_high=sim_res_ref_high,
    Sector_frequency=Sector_frequency,
    P=P,
)
yaw_opt["power"] = (["wt", "wd", "ws"], yaw_opt_power)
yaw_opt["lcoe"] = (["wt", "wd", "ws"], yaw_opt_lcoe)

In [ ]:
# rerun simulation for optimum
sim_res_opt, _, _ = utils.run_sim(
    wfm=utils.wfm_high,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=yaw_opt.lcoe.values.reshape(yaw_shape),
    ws=ws,
    wd=[wd],
    sim_res_ref=sim_res_ref_high.sel(wd=[wd]),
    Sector_frequency=Sector_frequency,
    P=P,
)
sim_res_opt

In [ ]:
# display comaprison of optimum to baseline
print(
    f"Lossless LCoE [USD/MWh]      : {sim_res_ref_lossless.lcoe_direction.sel(wd=wd).values:.3f}"
)
print(
    f"Baseline LCoE [USD/MWh]      : {sim_res_ref_high.lcoe_direction.sel(wd=wd).values:.3f}"
)
print(f"Optimum LCoE [USD/MWh]       : {sim_res_opt.lcoe_overall.values:.3f}")
print(
    f"Recovered LCoE [%] : {100-100*(sim_res_ref_lossless.lcoe_direction.sel(wd=wd).values - sim_res_opt.lcoe_overall.values)/(sim_res_ref_lossless.lcoe_direction.sel(wd=wd).values - sim_res_ref_high.lcoe_direction.sel(wd=wd).values):.2f}"
)
print(
    f"Lossless Capacity Factor [%] : {100*sim_res_ref_lossless.cap_fac_direction.sel(wd=wd).values:.3f}"
)
print(
    f"Baseline Capacity Factor [%] : {100*sim_res_ref_high.cap_fac_direction.sel(wd=wd).values:.3f}"
)
print(f"Optimum Capacity Factor [%]  : {100*sim_res_opt.cap_fac_overall.values:.3f}")
print(
    f"Recovered Capacity Factor [%] : {100-100*(sim_res_ref_lossless.cap_fac_direction.sel(wd=wd).values - sim_res_opt.cap_fac_overall.values)/(sim_res_ref_lossless.cap_fac_direction.sel(wd=wd).values - sim_res_ref_high.cap_fac_direction.sel(wd=wd).values):.2f}"
)